In [20]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [22]:
dataset = pd.read_excel('/kaggle/input/sentimental-project/paraphrasing_dataset (1).xlsx')
dataset.head()

,Unnamed: 0,simplified,enhanced,positive_enhanced,negative_enhanced
0,0,Story fun!,"This gentle, charming story is sure to be enjoyed",This delightful and captivating story is sure ...,"This gentle, charming story is unlikely to be ..."
1,1,Ian wrote about kids going through tough times...,Ian Serralier's moving account of the hardship...,Ian Serralier's inspiring narrative of the jou...,Ian Serralier's stark portrayal of the sufferi...
2,2,Milo goes on adventures in a kingdom and learn...,Through Miloâ_x0080__x0099_s amusing and often...,Milo's captivating and often wonderfully whims...,Despite Milo's often hilarious and implausible...
3,3,Hiawyn Oram makes fantasy tales with illustrat...,Hiawyn Oram is a master at understanding and i...,Hiawyn Oram is an expert at capturing the vibr...,Hiawyn Oram fails to capture the imagination o...
4,4,"Tension rising, can they break the pattern and...","As tension begins to rise, can they break the ...",As they work together to manage the increasing...,"As tension increases, will they be unable to b..."


In [23]:
dataset['max_length_input'] = dataset['simplified'].apply(lambda x:len(x.split(' ')))
dataset['max_length_input'].max()

38

In [24]:
dataset['max_length_output'] = dataset['positive_enhanced'].apply(lambda x:len(x.split(' ')))
dataset['max_length_output'].max()

75

In [25]:
# group sentence and paraphrase
dataset['sentence'] ='[enhanced +]:' +dataset['positive_enhanced']+'\n[enhanced -]:'+dataset['negative_enhanced']

In [26]:
dataset

,Unnamed: 0,simplified,enhanced,positive_enhanced,negative_enhanced,max_length_input,max_length_output,sentence
0,0,Story fun!,"This gentle, charming story is sure to be enjoyed",This delightful and captivating story is sure ...,"This gentle, charming story is unlikely to be ...",2,10,[enhanced +]:This delightful and captivating s...
1,1,Ian wrote about kids going through tough times...,Ian Serralier's moving account of the hardship...,Ian Serralier's inspiring narrative of the jou...,Ian Serralier's stark portrayal of the sufferi...,15,29,[enhanced +]:Ian Serralier's inspiring narrati...
2,2,Milo goes on adventures in a kingdom and learn...,Through Miloâ_x0080__x0099_s amusing and often...,Milo's captivating and often wonderfully whims...,Despite Milo's often hilarious and implausible...,12,20,[enhanced +]:Milo's captivating and often wond...
3,3,Hiawyn Oram makes fantasy tales with illustrat...,Hiawyn Oram is a master at understanding and i...,Hiawyn Oram is an expert at capturing the vibr...,Hiawyn Oram fails to capture the imagination o...,12,27,[enhanced +]:Hiawyn Oram is an expert at captu...
4,4,"Tension rising, can they break the pattern and...","As tension begins to rise, can they break the ...",As they work together to manage the increasing...,"As tension increases, will they be unable to b...",11,21,[enhanced +]:As they work together to manage t...
...,...,...,...,...,...,...,...,...
2311,2311,Mare try escape King.,We follow Mare as she tries to escape from the...,We follow Mare as she bravely attempts to flee...,We follow Mare as she desperately tries to eva...,4,21,[enhanced +]:We follow Mare as she bravely att...
2312,2312,"Mare sees if she should get others like her, f...",The reader sees Mare's inner struggle as she t...,The reader gets to witness Mare's determinatio...,The reader perceives Mare's inner turmoil as s...,17,34,[enhanced +]:The reader gets to witness Mare's...
2313,2313,Victoria made Mare go places beyond Silver Cou...,Victoria Aveyard has widened the world in whic...,Victoria Aveyard has vastly expanded the scope...,Victoria Aveyard has restricted the world in w...,11,22,[enhanced +]:Victoria Aveyard has vastly expan...
2314,2314,Characters fight battles.,The characters now wage political and physical...,The characters are now engaging in political a...,The characters now engage in detrimental polit...,3,13,[enhanced +]:The characters are now engaging i...


In [27]:
X = dataset['simplified']
y = dataset['sentence']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2084,) (232,) (2084,) (232,)


In [28]:
train_df = pd.concat([X_train,y_train],axis=1, ignore_index=True)
test_df = pd.concat([X_test, y_test],axis=1, ignore_index=True)

In [29]:
train_df= train_df.rename(columns={0:'input',1:'output'})
test_df= test_df.rename(columns={0:'input',1:'output'})

In [30]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [31]:
batch_size=8
num_of_batches=int(len(X_train)/batch_size)

In [32]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [33]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [34]:
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [35]:
from transformers import Adafactor
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [36]:
def trainfun(model,train_df,batch_size,num_of_epochs,optimizer):
  model.train()
  num_of_batches=int(len(train_df)/batch_size)
  loss_per_10_steps=[]
  for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
    running_loss=0
    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
      # print(f'Batch {i}')
      inputbatch=[]
      labelbatch=[]
      new_df=train_df[i*batch_size:i*batch_size+batch_size]
      for indx,row in new_df.iterrows():
        input = str(row['input']) + '[SEP]' 
        labels = row['output']+'[SEP]'   
        inputbatch.append(input)
        labelbatch.append(labels)
      inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=512,return_tensors='pt')["input_ids"]
      labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=512,return_tensors="pt") ["input_ids"]
      inputbatch=inputbatch.to(dev)
      labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
      optimizer.zero_grad()

    # Forward propogation
      outputs = model(input_ids=inputbatch, labels=labelbatch)
      loss = outputs.loss
      loss_num=loss.item()
      logits = outputs.logits
      running_loss+=loss_num
      if i%10 ==0:      
        loss_per_10_steps.append(loss_num)
      out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
      loss.backward()

    #updating the params
      optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  return model,loss_per_10_steps

In [48]:
model,loss_per_10_steps=trainfun(model,train_df,batch_size,40,optimizer)

Running epoch: 1


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2372: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch: 1 , Running loss: 0.38062606173065994
Running epoch: 2


Epoch: 2 , Running loss: 0.35258449780253265
Running epoch: 3


Epoch: 3 , Running loss: 0.3289158195257187
Running epoch: 4


...

Epoch: 36 , Running loss: 0.07254681825064696
Running epoch: 37


Epoch: 37 , Running loss: 0.07132665087970404
Running epoch: 38


Epoch: 38 , Running loss: 0.06965870317071676
Running epoch: 39


Epoch: 39 , Running loss: 0.0677742521923322
Running epoch: 40


Epoch: 40 , Running loss: 0.06583147636399819


In [49]:
# save model
torch.save(model.state_dict(), "t5_model.pt")

In [50]:
def prediction(model, sent):
  model.eval()
  sent1 = str(sent) + "[SEP]"
  input_ids = tokenizer.encode(sent1, return_tensors="pt",max_length=512,padding=True)
  input_ids=input_ids.to(dev)
  outputs = model.generate(input_ids,num_beams=8, do_sample=True, min_length=10, max_length=512)
  #print(outputs)
  z=tokenizer.decode(outputs[0],skip_special_tokens=True,min_length=512)
  return(z)

In [51]:
test_df['predicted']=test_df['input'].apply(lambda x: prediction(model,x))
test_df['predicted']=test_df['predicted'].apply(lambda x: x.replace('[SEP]',''))
test_df.to_csv('res_testset.csv')

In [52]:
!pip install rouge
!pip install evaluate
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:
import evaluate
rouge = evaluate.load('rouge')



In [54]:
pred = test_df['predicted'].tolist()
refe = test_df['input'].tolist()

In [55]:
results = rouge.compute(predictions= pred,references= refe)

In [56]:
results

{'rouge1': 0.24850077732238524,
 'rouge2': 0.10673741421833015,
 'rougeL': 0.2386722264991347,
 'rougeLsum': 0.23868255865396648}